In [1]:
""" Ingest Mapspam 2010 data into google bigquery.
-------------------------------------------------------------------------------

Ingest all variables except value of production to 
Google Bigquery. The script changes the column names for the crops to allow 
vertical stacking. 



Author: Rutger Hofste
Date: 20190904
Kernel: python36
Docker: rutgerhofste/gisdocker:ubuntu16.04

"""
TESTING = 0

SCRIPT_NAME = "Y2019M09D04_RH_Ingest_MAPSPAM_GBQ_V01"
OUTPUT_VERSION = 5

BQ_PROJECT_ID = "aqueduct30"
GCS_BUCKET = "aqueduct30_v01"
PREFIX = "Y2019M09D04_RH_Ingest_MAPSPAM_GBQ_V01"

MAPSPAM_CROPNAMES = "https://raw.githubusercontent.com/wri/MAPSPAM/master/metadata_tables/mapspam_names.csv"

gcs_input_path = "gs://{}/{}/input_V01".format(GCS_BUCKET,PREFIX)
gcs_output_path = "gs://{}/{}/output_V{:02.0f}/".format(GCS_BUCKET,PREFIX,OUTPUT_VERSION)
ec2_input_path = "/volumes/data/{}/input_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION)
ec2_output_path = "/volumes/data/{}/output_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION)

GBQ_OUTPUT_DATASET = "MAPSPAM_2010v10"

Download all csv files except production (has different schema) from 
http://mapspam.info/data/

Unzip and upload to Google Cloud Storage.

Rename:
spam2010v1r0_global_yield.csv -> spam2010V1r0_global_yield.csv  
spam2010v1r0_global_val_prod_agg.csv -> spam2010V1r0_global_val_prod_agg.csv

In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2019M09D05 UTC 10:02


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
import os
import subprocess
import pandas as pd
from tqdm import tqdm
from google.cloud import bigquery

In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/.google.json"
os.environ["GOOGLE_CLOUD_PROJECT"] = "aqueduct30"
client = bigquery.Client(project=BQ_PROJECT_ID)

In [5]:
!rm -r {ec2_input_path}
!rm -r {ec2_output_path}
!mkdir -p {ec2_input_path}
!mkdir -p {ec2_output_path}

rm: cannot remove '/volumes/data/Y2019M09D04_RH_Ingest_MAPSPAM_GBQ_V01/input_V05': No such file or directory
rm: cannot remove '/volumes/data/Y2019M09D04_RH_Ingest_MAPSPAM_GBQ_V01/output_V05': No such file or directory


In [6]:
!gsutil -m cp -r {gcs_input_path}/* {ec2_input_path}

Copying gs://aqueduct30_v01/Y2019M09D04_RH_Ingest_MAPSPAM_GBQ_V01/input_V01/spam2010V1r0_global_harv_area.csv/spam2010V1r0_global_H_TA.csv...
Copying gs://aqueduct30_v01/Y2019M09D04_RH_Ingest_MAPSPAM_GBQ_V01/input_V01/spam2010V1r0_global_harv_area.csv/spam2010V1r0_global_H_TH.csv...
Copying gs://aqueduct30_v01/Y2019M09D04_RH_Ingest_MAPSPAM_GBQ_V01/input_V01/spam2010V1r0_global_harv_area.csv/spam2010V1r0_global_H_TI.csv...
Copying gs://aqueduct30_v01/Y2019M09D04_RH_Ingest_MAPSPAM_GBQ_V01/input_V01/spam2010V1r0_global_harv_area.csv/spam2010V1r0_global_H_TL.csv...
Copying gs://aqueduct30_v01/Y2019M09D04_RH_Ingest_MAPSPAM_GBQ_V01/input_V01/spam2010V1r0_global_harv_area.csv/spam2010V1r0_global_H_TR.csv...
Copying gs://aqueduct30_v01/Y2019M09D04_RH_Ingest_MAPSPAM_GBQ_V01/input_V01/spam2010V1r0_global_harv_area.csv/spam2010V1r0_global_H_TS.csv...
Copying gs://aqueduct30_v01/Y2019M09D04_RH_Ingest_MAPSPAM_GBQ_V01/input_V01/spam2010V1r0_global_phys_area.csv/spam2010V1r0_global_A_TA.csv...
Copyin

In [7]:
variable_dict = {"yield":{"variable_short":"yield","shorthand":"Y"},
                 "production":{"variable_short":"prod","shorthand":"P"},
                 "harvested_area":{"variable_short":"harv_area","shorthand":"H"},
                 "physical_area":{"variable_short":"phys_area","shorthand":"A"}}
                 #"value_of_production":{"variable_short":"val_prod_agg","shorthand":"V_agg"}}

technologies =  ["A","I","H","L","S","R"] # see metadata

In [8]:
def load_df(variable_short,technology):
    folder_name = "spam2010V1r0_global_{}.csv".format(variable_short) 
    filename = "spam2010V1r0_global_{}_T{}.csv".format(shorthand,technology)
    input_path = "{}/{}/{}".format(ec2_input_path,folder_name,filename)
    df_raw = pd.read_csv(input_path,encoding="iso-8859-1")
    if TESTING:
        df = df_raw[0:100]
    else:
        df = df_raw
    return df

def rename_crop_columns(df,technology):
    """
    The csv files in Mapspam have the technology in the column names. The technology
    is also stored in the column tech_type and therefore redundant. It prevents 
    vertically stacking the data.
    
    Args:
        df(dataframe): Dataframe with old crop columns.
        technology(string):technology.
    Returs:
        df_renamed: Dataframe with renames columns
    
    """
    
    
    df_cropnames = pd.read_csv(MAPSPAM_CROPNAMES)
    new_crop_names = list(df_cropnames["SPAM_name"])
    old_crop_names = list(map(lambda x: x+"_{}".format(technology.lower()), new_crop_names))
    dictje = dict(zip(old_crop_names, new_crop_names))
    df_renamed = df.rename(columns=dictje)
    
    return df_renamed

In [9]:
for technology in tqdm(technologies):
    print(technology)
    for variable, values in variable_dict.items():
        print(variable)
        variable_short = values["variable_short"]
        shorthand = values["shorthand"]
        df = load_df(variable_short,technology)
        df_renamed = rename_crop_columns(df,technology)
        
        filename = "spam2010V1r0_global_{}_T{}.csv".format(shorthand,technology)
        output_path = "{}/{}".format(ec2_output_path,filename)
        
        df_renamed.to_csv(path_or_buf=output_path,
                          encoding="UTF-8")
        

        
        gbq_dataset_name = "MAPSPAM_2010v1r0"        
        table_name = technology
        destination_table= "{}.output_v{:02.0f}".format(gbq_dataset_name,OUTPUT_VERSION)
        
       
        df_renamed.to_gbq(project_id=BQ_PROJECT_ID,
                          destination_table=destination_table,
                          chunksize=100000,
                          if_exists="append")
                          
        


  0%|          | 0/6 [00:00<?, ?it/s]

A
physical_area



0it [00:00, ?it/s]
1it [00:04,  4.51s/it]
2it [00:29, 10.58s/it]
3it [01:15, 21.28s/it]
4it [01:40, 22.26s/it]
5it [02:05, 23.22s/it]
6it [02:30, 23.69s/it]
7it [03:02, 26.25s/it]
8it [03:38, 29.07s/it]
9it [03:59, 26.62s/it]

yield



0it [00:00, ?it/s]
1it [00:04,  4.56s/it]
2it [00:42, 14.56s/it]
3it [01:07, 17.82s/it]
4it [01:45, 23.73s/it]
5it [02:13, 25.05s/it]
6it [02:49, 28.46s/it]
7it [03:16, 27.82s/it]
8it [03:56, 31.59s/it]
9it [04:33, 33.03s/it]

...


0it [00:00, ?it/s]
1it [00:04,  4.39s/it]
2it [00:28, 10.25s/it]
3it [00:55, 15.43s/it]
4it [01:21, 18.61s/it]
5it [01:46, 20.45s/it]
6it [02:14, 22.68s/it]
7it [02:48, 25.97s/it]
8it [03:32, 31.37s/it]
9it [03:57, 29.50s/it]

harvested_area



0it [00:00, ?it/s]
1it [00:04,  4.35s/it]
2it [00:36, 12.63s/it]
3it [01:05, 17.65s/it]
4it [01:29, 19.49s/it]
5it [02:05, 24.60s/it]
6it [02:29, 24.42s/it]
7it [02:54, 24.30s/it]
8it [03:27, 27.06s/it]
9it [03:49, 25.60s/it]

production



0it [00:00, ?it/s]
1it [00:04,  4.44s/it]
2it [00:37, 12.92s/it]
3it [01:10, 18.92s/it]
4it [01:32, 19.96s/it]
5it [01:53, 20.42s/it]
6it [02:19, 21.90s/it]
7it [02:41, 22.06s/it]
8it [03:03, 22.12s/it]
100%|██████████| 6/6 [1:43:14<00:00, 1036.31s/it]


In [10]:
!gsutil -m cp -r {ec2_output_path} {gcs_output_path}

Copying file:///volumes/data/Y2019M09D04_RH_Ingest_MAPSPAM_GBQ_V01/output_V05/spam2010V1r0_global_H_TH.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2019M09D04_RH_Ingest_MAPSPAM_GBQ_V01/output_V05/spam2010V1r0_global_P_TH.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2019M09D04_RH_Ingest_MAPSPAM_GBQ_V01/output_V05/spam2010V1r0_global_H_TA.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2019M09D04_RH_Ingest_MAPSPAM_GBQ_V01/output_V05/spam2010V1r0_global_A_TI.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2019M09D04_RH_Ingest_MAPSPAM_GBQ_V01/output_V05/spam2010V1r0_global_H_TR.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2019M09D04_RH_Ingest_MAPSPAM_GBQ_V01/output_V05/spam2010V1r0_global_Y_TI.csv [Content-Type=text/csv]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite

In [11]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

1:43:49.833856


previous run:  
1:43:49.833856
